# 00 - Instalando pacotes

In [ ]:
! pip install requests pandas ipeadatapy statsforecast xgboost tensorflow scikit-learn matplotlib prophet

# 01 - Import de bibliotecas

In [ ]:
from ipeadatapy import list_series
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from statsforecast import StatsForecast
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import joblib

# 02 - ETL

In [ ]:
# Código da série no IPEA
codigo_serie = 'EIA366_PBRENT366'  #Preço por barril do petróleo bruto Brent (FOB) (EIA366_PBRENT366)

# URL da API
url = f"http://ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo_serie}')"

# GET de dados
response = requests.get(url)

# convertendo retorno para JSON
dados_json = response.json()

# Extraindo dados da chave 'value'
dados = dados_json['value']

# Criando DataFrame com os dados
df = pd.DataFrame(dados)

# Selecionando as colunas de interesse
df = df[['VALDATA', 'VALVALOR']]

In [ ]:
print(df)

In [ ]:
# Renommeando as colunas
df.columns = ['Data', 'Valor']

In [ ]:
# cortando a string de data
df['Data'] = df['Data'].str[:10]

In [ ]:
# Alterando formato para data
df['Data'] = pd.to_datetime(df['Data'])

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
# Verificar valores ausentes
print("Valores ausentes:\n", df.isnull().sum())

In [ ]:
# Dropando Valores Ausentes
df = df.dropna()

In [ ]:
# Verificar duplicatas
print("Duplicadas:", df.duplicated().sum())

In [ ]:
# Ordenar por data
df = df.sort_values("Data")

In [ ]:
# Remover duplicatas, se houver
df = df.drop_duplicates()

In [ ]:
# Exibir amostra final
print(df.tail(10))

# 03 - EDA (Exploratory data Analysis)

In [ ]:
# Plot da Série Geral

plt.figure(figsize=(14, 6))
sns.lineplot(x="Data", y="Valor", data=df)
plt.title("Preço histórico do Petróleo Brent (USD)")
plt.xlabel("Data")
plt.ylabel("Preço (US$)")
plt.grid(True)
plt.show()

In [ ]:
# # Volatilidade do preço do petróleo (com base em 30 dias)

# df["Retorno (%)"] = df["Valor"].pct_change() * 100

# # Volatilidade (rolling std de 30 dias)
# df["Volatilidade (%)"] = df["Retorno (%)"].rolling(30).std()

# # Visualizar
# plt.figure(figsize=(14, 5))
# sns.lineplot(x="Data", y="Volatilidade (%)", data=df)
# plt.title("Volatilidade (30 dias) do preço do Brent")
# plt.grid(True)
# plt.show()

In [ ]:
# Destacando diferentes períodos

'''
destacando diversos períodos dentro da série histórico como:

  - Em 2003 a Invasão do iraque (um dos países membros da OPEP) deveria em tese causar
  algum impacto na produção de petróleo que por sua vez seria notado na série histórica de preços

  - Em 2008 a crise dos subprimes nos Estados Unidos afetou em cadeia todo a economia america e também mundial,
  era esperada uma diminuição do consumo geral neste período.

  - Em 2020 a pandemia Global do Corona virus impactou diretamente a demanda por produtos e commodities,
  o impacto esperado era de uma queda no consumo e consequentemente no preço.

  - Em 2022 ocorreu a invasão da ucrânia por porte da Rússia, o que causou caos e dificuldades nos demais países UE
  Era esperado uma elevação no consumo dados que a matriz energética de muitos desses paises depende de petróleo
  e derivados no perído de inverno.

'''

plt.figure(figsize=(14, 6))
sns.lineplot(x="Data", y="Valor", data=df)

# 01 - Invsão Iraque
plt.axvline(pd.Timestamp("2003-03-20"), color="black", linestyle="--")
plt.text(pd.Timestamp("2003-03-20"), 95, "Invasão do Iraque", color="black")
# 02 - Crise 2008
plt.axvspan("2008-07-01", "2009-01-01", color="red", alpha=0.2)
plt.text(pd.Timestamp("2008-07-01"), 140, "Crise de 2008", color="red")
# 03 - Covid
plt.axvspan("2020-03-01", "2020-05-01", color="orange", alpha=0.2)
plt.text(pd.Timestamp("2020-03-10"), 25, "COVID-19", color="orange")
# 04 - Invsão Ucrania
plt.axvline(pd.Timestamp("2022-02-24"), color="purple", linestyle="--")
plt.text(pd.Timestamp("2022-02-25"), 115, "Invasão da Ucrânia", color="purple")
plt.title("Série Histórica com fatos destacados")
plt.grid(True)
plt.show()

In [ ]:
# Isso insere todas as datas (com NaN nos dias faltantes)
df_decomp = df.set_index("Data").asfreq("D")

In [ ]:
# Interpolação linear para preencher valores faltantes
df_decomp["Valor"] = df_decomp["Valor"].interpolate(method="linear")

In [ ]:
# Decomposição da série temporal

decomp = seasonal_decompose(df_decomp["Valor"], model="multiplicative", period=365)

# Captura o objeto da figura
fig = decomp.plot()
fig.set_size_inches(14, 10)  # Largura x Altura em polegadas

plt.suptitle("Decomposição da série temporal do preço do Brent", fontsize=16)
plt.tight_layout()
plt.show()

# 04 - Modelos

In [ ]:
def avaliar_modelo(y_real, y_pred, nome_modelo):
    mae = mean_absolute_error(y_real, y_pred)
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))
    r2 = r2_score(y_real, y_pred)

    return {
        "Modelo": nome_modelo,
        "MAE": round(mae, 3),
        "RMSE": round(rmse, 3),
        "R²": round(r2, 3)
    }

## 4.1 - Naive

In [ ]:
# Usar a versão com índice diário e valores interpolados
df_naive = df.set_index("Data").asfreq("D")
df_naive["Valor"] = df_naive["Valor"].interpolate(method="linear") # não será feita interpolação

# Separar treino e teste
train = df_naive[:-30]   # Ex: tudo menos os últimos 30 dias
test = df_naive[-30:]    # últimos 30 dias

In [ ]:
# Previsão naive: repete o último valor do treino
y_pred = [train["Valor"].iloc[-1]] * len(test)
y_true = test["Valor"].values

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(train.index[-60:], train["Valor"].iloc[-60:], label="Treino (últimos 60 dias)")
plt.plot(test.index, y_true, label="Real (teste)", color="green")
plt.plot(test.index, y_pred, label="Previsão Naive", linestyle="--", color="orange")
plt.title("Previsão com Modelo Naive")
plt.xlabel("Data")
plt.ylabel("Preço do Brent (US$)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
resultado_naive = avaliar_modelo(y_true, y_pred, "Naive")

## 4.2 - XGBoost

In [ ]:
# Reindexar e interpolar
df_xgb = df.set_index("Data").asfreq("D")
df_xgb["Valor"] = df_xgb["Valor"].interpolate(method="linear")

In [ ]:
# Criar features de lag
df_xgb["lag1"] = df_xgb["Valor"].shift(1)
df_xgb["lag2"] = df_xgb["Valor"].shift(2)
df_xgb["lag7"] = df_xgb["Valor"].shift(7)

In [ ]:
# Rolling mean
df_xgb["roll_mean_3"] = df_xgb["Valor"].rolling(3).mean()
df_xgb["roll_mean_7"] = df_xgb["Valor"].rolling(7).mean()

In [ ]:
# Features de data
df_xgb["day"] = df_xgb.index.day
df_xgb["month"] = df_xgb.index.month
df_xgb["weekday"] = df_xgb.index.weekday

In [ ]:
# Remover valores nulos (primeiras linhas com lag e rolling)
df_xgb.dropna(inplace=True)

In [ ]:
X = df_xgb.drop("Valor", axis=1)
y = df_xgb["Valor"]

# Usar últimos 30 dias como teste
X_train, X_test = X[:-30], X[-30:]
y_train, y_test = y[:-30], y[-30:]

In [ ]:
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
resultado_xgb = avaliar_modelo(y_true, y_pred, "XGBoost")

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(y_test.index, y_test, label="Real")
plt.plot(y_test.index, y_pred, label="Previsão XGBoost", linestyle="--")
plt.title("Previsão com XGBoost")
plt.xlabel("Data")
plt.ylabel("Preço do Brent (US$)")
plt.legend()
plt.grid(True)
plt.show()

##

## 4.3 - LSTM

In [ ]:
# 1. Base com frequência diária e preenchida
df_lstm = df.set_index("Data").asfreq("D")
df_lstm["Valor"] = df_lstm["Valor"].interpolate(method="linear")

In [ ]:
# 2. Normalizar
scaler = MinMaxScaler()
df_lstm["Valor_scaled"] = scaler.fit_transform(df_lstm[["Valor"]])

In [ ]:
# 3. Criar janelas de tempo
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size:i])
        y.append(data[i])
    return np.array(X), np.array(y)

window_size = 30  # Ex: usar os últimos 30 dias para prever o próximo
series = df_lstm["Valor_scaled"].values
X, y = create_sequences(series, window_size)

In [ ]:
# 4. Dividir treino e teste
split = -30  # últimos 30 dias para teste
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# 5. Redimensionar para [amostras, time_steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, activation="relu", input_shape=(window_size, 1)))
lstm_model.add(Dense(1))  # Previsão de 1 valor

lstm_model.compile(optimizer="adam", loss="mse")

# Early stopping para evitar overfitting
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = lstm_model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=16,
    callbacks=[es],
    verbose=1
)

In [ ]:
y_pred = lstm_model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
resultado_lstm = avaliar_modelo(y_test_rescaled, y_pred_rescaled, "LSTM")

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_lstm.index[-30:], y_test_rescaled, label="Real")
plt.plot(df_lstm.index[-30:], y_pred_rescaled, label="Previsão LSTM", linestyle="--")
plt.title("Previsão com LSTM")
plt.xlabel("Data")
plt.ylabel("Preço do Brent (US$)")
plt.legend()
plt.grid(True)
plt.show()

## 4.4 - Prophet

In [ ]:
df_prophet = df.rename(columns={'Data': 'ds', 'Valor': 'y'})

In [ ]:
# Separar os últimos 730 dias para teste
periodo = 730
data_final = df_prophet['ds'].max()
data_inicio = df_prophet['ds'].sort_values().iloc[-periodo]

#treino e teste
train_data = df_prophet[df_prophet['ds'] < data_inicio].copy()
test_data = df_prophet[df_prophet['ds'] >= data_inicio].copy()

In [ ]:
prophet_model = Prophet(daily_seasonality=True)
prophet_model.add_seasonality(name='weekly', period=7, fourier_order=5)
prophet_model.fit(train_data)

In [ ]:
# Criar datas futuras incluindo o período de teste
fut = prophet_model.make_future_dataframe(periods=30)

# Gerar previsões
previsao = prophet_model.predict(fut)

# Extrair previsões apenas para o período de teste
prev_test = previsao[previsao['ds'].isin(test_data['ds'])][['ds', 'yhat']].reset_index(drop=True)
real_test = test_data[['ds', 'y']].reset_index(drop=True)

In [ ]:
comparacao = pd.merge(real_test, prev_test, on='ds', how='inner')

In [ ]:
fig = prophet_model.plot(previsao, figsize=(20, 6))
plt.plot(test_data['ds'], test_data['y'], '.r', label='Valores Reais')
plt.title(f'Previsão dos próximos dias')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
resultado_prophet = avaliar_modelo(comparacao['y'], comparacao['yhat'], "prophet")

# 05 - Avaliação dos modelos e Export

In [ ]:
resultados = pd.DataFrame([
    resultado_naive,
    resultado_xgb,
    resultado_lstm,
    resultado_prophet
])

print(resultados)

In [ ]:
# Export do modelo escolhido

joblib.dump(xgb_model, 'xgb_petroleo_model.pkl')
print('Modelo XGBoost exportado como xgb_petroleo_model.pkl')